In [1]:
import pandas as pd
# df_item = pd.read_csv('./tianchi_fresh_comp_train_item.csv')
df_user = pd.read_csv('./tianchi_fresh_comp_train_user.csv')

根据日期范围11.18~12.18分割数据集：

- 抽取11.18~12.17的数据作为训练集
- 抽取12.18的数据作为测试集
- 制作标签：
- 其中11.18~12.17的数据集的标签用这段期间内是否曾经过买过来拟合买/不买

用人工规则创作特征：
脑洞越大，越贴近业务实际越好

- 商品总浏览次数
- 商品前一天，也就是17号有没有加入购物车：前一天加入购物车的最有可能被买
- 商品有没有曾经（包括前一天）被加购物车，曾经被加购物车的商品最有可能被买，即便不是前一天加进去的
- 商品有没有曾经被收藏：有收藏的最有可能被买
- 商品总浏览次数：浏览的越多越有可能被买
- 该用户总购买次数：用户购买东西越多，用户购买力越强
- 该商品被购买次数：商品被买的越多，该商品越畅销

训练集：

| 用户商品id|总收藏次数|在购物车次数|16号加购物车数量|总浏览次数|用户购买次数|商品被买次数|17号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|否

测试集

| 用户商品id|总收藏次数|在购物车次数|17号加购物车数量|总浏览次数|用户购买次数|商品被买次数|18号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|是

要提交的最终模型训练集:

|用户商品id|总收藏次数|在购物车次数|18号加购物车数量|总浏览次数|用户购买次数|商品被买次数|19号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|待预测

最后根据得到的预测结果筛选出用户商品id和购买为**是**的列，整理成用户-商品id的对

具体思路：

- 用户购买数:用`groupby` 将用户id作为一个键，然后`count(behavior_type)` 然后再取behavior_type=4的count值

- 商品被购买次数:用`groupby` 将商品id作为一个键，然后`count(behavior_type)` 然后再取behavior_type=4的count值

- 总浏览次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=1的count值

- 总收藏次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=2的count值

- 在购物车次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=3的count值

- 前一天在购物车次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=3 且 time = 前一天日期 的count值

不会groupby和count的[拿去](https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-object-to-dataframe)

In [2]:
#删掉我用不上的地理哈希值
del df_user['user_geohash']

In [3]:
df_user.head()

,user_id,item_id,behavior_type,item_category,time
0,10001082,285259775,1,4076,2014-12-08 18
1,10001082,4368907,1,5503,2014-12-12 12
2,10001082,4368907,1,5503,2014-12-12 12
3,10001082,53616768,1,9762,2014-12-02 15
4,10001082,151466952,1,5232,2014-12-12 11


In [4]:
df_user.shape

(23291027, 5)

In [5]:
# 用户购买次数
df_user_behavior_count = pd.DataFrame({'count_user_buy' : df_user[['user_id','behavior_type']].groupby(['user_id','behavior_type']).size()}).reset_index()
# 取出behavior_type=4的行
df_user_buy_count = df_user_behavior_count.loc[df_user_behavior_count['behavior_type'] == 4]
# 删掉behavior_type以便用来join
del df_user_buy_count['behavior_type']
#与原来的user表join在一起:
# eg: caller.join(other.set_index('key'), on='key')
df_user = df_user.join(df_user_buy_count.set_index('user_id'), on='user_id')
# 删掉这两个临时数据表
del df_user_buy_count
del df_user_behavior_count
df_user.head()

,user_id,item_id,behavior_type,item_category,time,count_user_buy
0,10001082,285259775,1,4076,2014-12-08 18,4.0
1,10001082,4368907,1,5503,2014-12-12 12,4.0
2,10001082,4368907,1,5503,2014-12-12 12,4.0
3,10001082,53616768,1,9762,2014-12-02 15,4.0
4,10001082,151466952,1,5232,2014-12-12 11,4.0


In [6]:
# 商品被购买次数:
df_item_behavior_count = pd.DataFrame({'count_item_bought' : df_user[['item_category','behavior_type']].groupby(['item_category','behavior_type']).size()}).reset_index()
df_item_bought_count =  df_item_behavior_count.loc[df_item_behavior_count['behavior_type'] == 4]
del df_item_bought_count['behavior_type']
df_user = df_user.join(df_item_bought_count.set_index('item_category'), on='item_category')
del df_item_bought_count
del df_item_behavior_count
df_user.head()

,user_id,item_id,behavior_type,item_category,time,count_user_buy,count_item_bought
0,10001082,285259775,1,4076,2014-12-08 18,4.0,44.0
1,10001082,4368907,1,5503,2014-12-12 12,4.0,690.0
2,10001082,4368907,1,5503,2014-12-12 12,4.0,690.0
3,10001082,53616768,1,9762,2014-12-02 15,4.0,74.0
4,10001082,151466952,1,5232,2014-12-12 11,4.0,3049.0


In [7]:
# 在购物车次数
df_user_item_behavior_count = pd.DataFrame({'count_behavior' : df_user[['user_id','item_category','behavior_type']].groupby(['user_id','item_category','behavior_type']).size()}).reset_index()

In [8]:
df_user_item_behavior_count

,user_id,item_category,behavior_type,count_behavior
0,492,280,1,5
1,492,451,1,2
2,492,675,1,2
3,492,2170,1,38
4,492,2170,3,1
5,492,2298,1,2
6,492,2413,1,5
7,492,2926,1,3
8,492,3424,1,2
9,492,4640,1,2


In [9]:
# 某个商品子类被某个用户浏览的总次数
df_user_item_look_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 1]
df_user_item_look_count = df_user_item_look_count.rename(columns = {'count_behavior':'count_look'})
del df_user_item_look_count['behavior_type']
# df_user_item_look_count
# 
# del df_user_item_look_count

In [10]:
# 某个商品子类被某个用户收藏的总次数
df_user_item_favor_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 2]
df_user_item_favor_count = df_user_item_favor_count.rename(columns = {'count_behavior':'count_favor'})
del df_user_item_favor_count['behavior_type']
# df_user_item_favor_count
# 
# del df_user_item_favor_count

In [13]:
# 某个商品子类被某个用户添加购物车的次数
df_user_item_addcart_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 3]
df_user_item_addcart_count = df_user_item_addcart_count.rename(columns = {'count_behavior':'count_addcart'})
del df_user_item_addcart_count['behavior_type']
# df_user_item_favor_count
# 
# del df_user_item_addcart_count

In [16]:
df_user_item_look_count

,user_id,item_category,count_look
0,492,280,5
1,492,451,2
2,492,675,2
3,492,2170,38
5,492,2298,2
6,492,2413,5
7,492,2926,3
8,492,3424,2
9,492,4640,2
10,492,4676,3


In [18]:
del df_user['item_id']
del df_user['behavior_type']
del df_user['time']

In [20]:
df_user = df_user.drop_duplicates()

In [21]:
df_user

,user_id,item_category,count_user_buy,count_item_bought
0,10001082,4076,4.0,44.0
1,10001082,5503,4.0,690.0
3,10001082,9762,4.0,74.0
4,10001082,5232,4.0,3049.0
7,10001082,10894,4.0,1678.0
8,10001082,6513,4.0,2195.0
10,10001082,2825,4.0,1154.0
12,10001082,3200,4.0,37.0
13,10001082,10576,4.0,21.0
17,10001082,7079,4.0,119.0


In [22]:
df_user = pd.merge(df_user, df_user_item_look_count, how='left', left_on=['user_id','item_category'], right_on=['user_id','item_category'])
df_user = pd.merge(df_user, df_user_item_favor_count, how='left', left_on=['user_id','item_category'], right_on=['user_id','item_category'])
df_user = pd.merge(df_user, df_user_item_addcart_count, how='left', left_on=['user_id','item_category'], right_on=['user_id','item_category'])

In [23]:
df_user

,user_id,item_category,count_user_buy,count_item_bought,count_look,count_favor,count_addcart
0,10001082,4076,4.0,44.0,1.0,NaN,NaN
1,10001082,5503,4.0,690.0,6.0,NaN,NaN
2,10001082,9762,4.0,74.0,7.0,NaN,NaN
3,10001082,5232,4.0,3049.0,19.0,NaN,NaN
4,10001082,10894,4.0,1678.0,10.0,NaN,NaN
5,10001082,6513,4.0,2195.0,11.0,NaN,NaN
6,10001082,2825,4.0,1154.0,22.0,NaN,NaN
7,10001082,3200,4.0,37.0,2.0,NaN,NaN
8,10001082,10576,4.0,21.0,45.0,NaN,NaN
9,10001082,7079,4.0,119.0,3.0,NaN,NaN


In [26]:
df_user = df_user.fillna(0)

In [27]:
df_user

,user_id,item_category,count_user_buy,count_item_bought,count_look,count_favor,count_addcart
0,10001082,4076,4.0,44.0,1.0,0.0,0.0
1,10001082,5503,4.0,690.0,6.0,0.0,0.0
2,10001082,9762,4.0,74.0,7.0,0.0,0.0
3,10001082,5232,4.0,3049.0,19.0,0.0,0.0
4,10001082,10894,4.0,1678.0,10.0,0.0,0.0
5,10001082,6513,4.0,2195.0,11.0,0.0,0.0
6,10001082,2825,4.0,1154.0,22.0,0.0,0.0
7,10001082,3200,4.0,37.0,2.0,0.0,0.0
8,10001082,10576,4.0,21.0,45.0,0.0,0.0
9,10001082,7079,4.0,119.0,3.0,0.0,0.0
